In [8]:
@time [1/number for number in 1:10]

  0.046238 seconds (56.18 k allocations: 2.799 MiB, 19.07% gc time)


10-element Array{Float64,1}:
 1.0                
 0.5                
 0.3333333333333333 
 0.25               
 0.2                
 0.16666666666666666
 0.14285714285714285
 0.125              
 0.1111111111111111 
 0.1                

In [9]:
@time [log2(number) for number in 1:10]

  0.000014 seconds (7 allocations: 384 bytes)


10-element Array{Float64,1}:
 0.0              
 1.0              
 1.584962500721156
 2.0              
 2.321928094887362
 2.584962500721156
 2.807354922057604
 3.0              
 3.169925001442312
 3.321928094887362

@which log2(2)

In [205]:
?Expr

search: Expr export my_expression nextprod expanduser exp2 exp expm1 exp10



No documentation found.

# Summary

```
mutable struct Expr <: Any
```

# Fields

```
head :: Symbol
args :: Array{Any,1}
```


In [13]:
x = 2

2

In [14]:
:x

:x

In [15]:
:(2 * 2)

:(2 * 2)

In [16]:
quote
    2 * 2
end

quote
    #= In[16]:2 =#
    2 * 2
end

In [23]:
bitstring(4)

"0000000000000000000000000000000000000000000000000000000000000100"

In [32]:
[1,2,3][end-1:end]

2-element Array{Int64,1}:
 2
 3

In [42]:
my_expression = quote
    for i = 0:8
        println(bitstring(i)[end-4:end])
    end
end

quote
    #= In[42]:2 =#
    for i = 0:8
        #= In[42]:3 =#
        println((bitstring(i))[end - 4:end])
    end
end

In [43]:
typeof(my_expression)

Expr

In [44]:
eval(my_expression)

00000
00001
00010
00011
00100
00101
00110
00111
01000


In [48]:
fieldnames(Expr)

(:head, :args)

In [49]:
my_expression.head

:block

In [50]:
my_expression.args

2-element Array{Any,1}:
 :(#= In[42]:2 =#)                                                                   
 :(for i = 0:8
      #= In[42]:3 =#
      println((bitstring(i))[end - 4:end])
  end)

In [54]:
my_expression.args[2].head

:for

In [56]:
my_expression.args[2].args[1]

:(i = 0:8)

In [64]:
my_favorite_number_other_than_eight = 7

7

In [65]:
my_expression.args[2].args[1] = :(i = 0:$my_favorite_number_other_than_eight)

:(i = 0:7)

In [66]:
eval(my_expression)

00000
00001
00010
00011
00100
00101
00110
00111


In [69]:
for (line_number, expression_argument) in enumerate(my_expression.args)
    println(line_number, ":: ", expression_argument)
end

1:: #= In[42]:2 =#
2:: for i = 0:7
    #= In[42]:3 =#
    println((bitstring(i))[end - 4:end])
end


In [70]:
dump(my_expression)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[42]
    2: Expr
      head: Symbol for
      args: Array{Any}((2,))
        1: Expr
          head: Symbol =
          args: Array{Any}((2,))
            1: Symbol i
            2: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol :
                2: Int64 0
                3: Int64 7
        2: Expr
          head: Symbol block
          args: Array{Any}((2,))
            1: LineNumberNode
              line: Int64 3
              file: Symbol In[42]
            2: Expr
              head: Symbol call
              args: Array{Any}((2,))
                1: Symbol println
                2: Expr


In [72]:
:(x = $(sin(1)) )

:(x = 0.8414709848078965)

In [83]:
function my_function(n, x...)
    x[n]
end

my_function (generic function with 2 methods)

In [84]:
my_function(1, 2, 3)

2

In [85]:
my_function(3, 2, 1)

BoundsError: BoundsError: attempt to access (2, 1)
  at index [3]

In [145]:
?@macroexpand

```
@macroexpand
```

Return equivalent expression with all macros removed (expanded).

There are differences between `@macroexpand` and [`macroexpand`](@ref).

  * While [`macroexpand`](@ref) takes a keyword argument `recursive`, `@macroexpand`

is always recursive. For a non recursive macro version, see [`@macroexpand1`](@ref).

  * While [`macroexpand`](@ref) has an explicit `module` argument, `@macroexpand` always

expands with respect to the module in which it is called. This is best seen in the following example:

```julia-repl
julia> module M
           macro m()
               1
           end
           function f()
               (@macroexpand(@m),
                macroexpand(M, :(@m)),
                macroexpand(Main, :(@m))
               )
           end
       end
M

julia> macro m()
           2
       end
@m (macro with 1 method)

julia> M.f()
(1, 1, 2)
```

With `@macroexpand` the expression expands where `@macroexpand` appears in the code (module `M` in the example). With `macroexpand` the expression expands in the module given as the first argument.


In [120]:
module M
    function macro1()
        1
    end

    macro macro2()
        2
    end
    
    macro macro3()
        :(@macro2())
    end

    macro macro4()
        macro1
    end

    macro macro5()
        macro2
    end
end

Main.M

In [123]:
M.macro1

macro1 (generic function with 1 method)

In [146]:
M.@macro2

2

In [125]:
@M.macro2

2

In [126]:
@M.macro2()

2

In [170]:
M.@macro4

macro1 (generic function with 1 method)

In [162]:
M.@macro5

LoadError: UndefVarError: macro2 not defined

In [107]:
macroexpand(M, :( @macro2() ) )

42

In [104]:
macro macro1()
    123
end

@macro1 (macro with 1 method)

In [101]:
@macroexpand @macro1()

42

In [102]:
@macro1()

42

In [141]:
macro twostep(arg)
   println("I execute at parse time. The argument is: ", arg)
   return :(println("I execute at runtime. The argument is: ", $arg))
end


@twostep (macro with 1 method)

In [142]:
@twostep(1)

I execute at parse time. The argument is: 1
I execute at runtime. The argument is: 1


In [171]:
@macroexpand @twostep(:(1, 2, 3))

I execute at parse time. The argument is: $(Expr(:quote, :((1, 2, 3))))


:((Main.println)("I execute at runtime. The argument is: ", $(Expr(:copyast, :($(QuoteNode(:((1, 2, 3)))))))))

In [172]:
ex = macroexpand(Main, :(@twostep :(1, 2, 3)) );

I execute at parse time. The argument is: $(Expr(:quote, :((1, 2, 3))))


In [176]:
@twostep :(1, 2, 3)

I execute at parse time. The argument is: $(Expr(:quote, :((1, 2, 3))))
I execute at runtime. The argument is: (1, 2, 3)


In [178]:
@twostep(1,2,3)

LoadError: MethodError: no method matching @twostep(::LineNumberNode, ::Module, ::Int64, ::Int64, ::Int64)
Closest candidates are:
  @twostep(::LineNumberNode, ::Module, ::Any) at In[141]:2

In [177]:
@twostep((1,2,3))

I execute at parse time. The argument is: (1, 2, 3)
I execute at runtime. The argument is: (1, 2, 3)


In [175]:
@twostep 1, 2, 3

I execute at parse time. The argument is: (1, 2, 3)
I execute at runtime. The argument is: (1, 2, 3)


In [174]:
@twostep (1, 2, 3)

I execute at parse time. The argument is: (1, 2, 3)
I execute at runtime. The argument is: (1, 2, 3)


In [179]:
function twostep(arg)
    println("I execute at parse time. The argument is: ", arg)
    return :(println("I execute at runtime. The argument is: ", $arg))
end

twostep (generic function with 1 method)

In [182]:
twostep(:((1,2,3)))

I execute at parse time. The argument is: (1, 2, 3)


:(println("I execute at runtime. The argument is: ", (1, 2, 3)))

In [203]:
print(:(1,2,3))

(1, 2, 3)

In [183]:
print(:(sin(3)))

sin(3)

In [184]:
print(sin(3))

0.1411200080598672

In [193]:
x = 10
print(:(sin($(x+3))))

sin(13)

In [200]:
print("sin($(x+3))")

sin(13)

In [210]:
dump(:(a in (1,2,3)))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol in
    2: Symbol a
    3: Expr
      head: Symbol tuple
      args: Array{Any}((3,))
        1: Int64 1
        2: Int64 2
        3: Int64 3


In [211]:
dump(:(a in $(1,2,3)))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol in
    2: Symbol a
    3: Tuple{Int64,Int64,Int64}
      1: Int64 1
      2: Int64 2
      3: Int64 3


In [212]:
dump(:(a in $(:(1,2,3))))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol in
    2: Symbol a
    3: Expr
      head: Symbol tuple
      args: Array{Any}((3,))
        1: Int64 1
        2: Int64 2
        3: Int64 3


In [218]:
Meta.show_sexpr(a in (1,2,3))

UndefVarError: UndefVarError: a not defined

In [217]:
Meta.show_sexpr(:(a in (1,2,3)))

(:call, :in, :a, (:tuple, 1, 2, 3))

In [219]:
Meta.show_sexpr(:(a in $(1,2,3)))

(:call, :in, :a, (1, 2, 3))

In [220]:
macro sayhello(name)
    return :( println("Hello, ", $name) )
end

@sayhello (macro with 1 method)

In [223]:
@macroexpand @sayhello name

:((Main.println)("Hello, ", Main.name))

In [224]:
@macroexpand @sayhello "name"

:((Main.println)("Hello, ", "name"))

In [225]:
@macroexpand @sayhello :("name")

:((Main.println)("Hello, ", $(QuoteNode("name"))))

In [228]:
macro show_arguments(arguments)
    show(arguments)
end

@show_arguments (macro with 1 method)

In [232]:
x = 2

2

In [235]:
@show_arguments sin(x)

:(sin(x))

In [234]:
@show_arguments :(sin(x))

:($(Expr(:quote, :(sin(x)))))

In [236]:
@macroexpand @assert true == false

:(if true == false
      nothing
  else
      (Base.throw)((Base.AssertionError)("true == false"))
  end)

In [243]:
b"100"

3-element Base.CodeUnits{UInt8,String}:
 0x31
 0x30
 0x30

In [246]:
Array(5:-1:1)

5-element Array{Int64,1}:
 5
 4
 3
 2
 1

In [1]:
macro nest(n::Int, f) 
    expression = :x
    for _ = 1 : n
       expression = Expr(:call, f, expression)
    end
    :(x -> $expression)
end

@nest (macro with 1 method)

In [2]:
(@nest 3 sin)(1)

0.6784304773607402

In [6]:
using BenchmarkTools

In [7]:
@btime (@nest 3 sin)(1)

  1.537 ns (0 allocations: 0 bytes)


0.6784304773607402

In [9]:
function nest(n::Int, f)
    function nested_f(x)
        argument = x
        for _ in 1:n
            argument= f(argument)
        end
        return argument
    end
    return nested_f
end

nest (generic function with 1 method)

In [12]:
@btime nest(3, sin)(1)

  48.157 ns (0 allocations: 0 bytes)


0.6784304773607402